# Bias Correction Methods
## Overview
We use a variation of the modified equidistant quantile matching (EDCDFm) algorithm ([Li et al. 2010](http://dx.doi.org/10.1029/2009JD012882)) of ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)) to bias correct CMIP5 projections for the Red River, Vietnam. Key differences from ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)) include:

* We apply EDCDFm to daily temperature and precipitation anomalies instead of absolute values and bias correct the mean separately. Unlike ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)), this allows the EDCDFm deltas for precipitation to be additive instead of multiplicative and avoids multiplicative instabilities. We calculate temperature anomalies as an additive difference from the mean and temperature anomalies as a ratio of the mean.   
* We only apply EDCDFm over wet days when bias correcting precipitation. Unlike ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)), this allows a model-predicted increase in number of wet days to be preserved and keeps models with too many historical dry days from being changed to all wet days over the historical period when biased corrected.
* We do not apply the frequency-dependent bias correction method described in section 4 of ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)).
* If bias correction is applied without a moving time window, it will correct extremes, but not the annual cycle. Conversely, if bias correction is applied over a small moving window, it will correct the annual cycle, but may not correctly represent extremes. This is especially true for the EDCDFm method. To balance the correction of extremes and the annual cycle, ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)) use a “preconditioning” step and an iterative application of EDCDFm over different moving window sizes (section 5). We use an alternative method. We first bias correct with EDCDFm using a 31-day moving window. We then bias correct with EDCDFm without a moving window. Finally, we bias correct the values from the 31-day moving window bias correction using those from the bias correction without a moving window.

## Symbols

* $W_t$ precipitation wet day threshold
* $K$ mean value correction factor
* $O_{base}$ observations for historical base time period
* $M_{base}$ modeled values for historical base time period
* $M_{fut}$ modeled values for time period to bias correct
* $M_{bc}$ bias corrected modeled values for time period to bias correct
* $wetO_{base}$ wet day precipitation observations for historical base time period 
* $wetM_{base}$ modeled wet day daily precipitation values for base time period
* $wetM_{fut}$ modeled wet day daily precipitation values for time period to bias correct
* $wetM_{bc}$ bias corrected modeled wet day daily precipitation values for time period to bias
* $\delta wetO_{base}$ observed wet day precipitation anomalies for historical base time period
* $\delta wetM_{base}$ modeled wet day precipitation anomalies for base time period
* $\delta wetM_{fut}$ modeled wet day precipitation anomalies for  time period to bias correct
* $\delta wetM_{bc}$ bias corrected wet day precipitation anomalies for time period to bias correct
* $\overline{wetO_{base}}$ mean of observed wet day precipitation for historical base time period
* $\overline{wetM_{base}}$ mean of modeled wet day precipitation for base time period
* $\overline{wetM_{fut}}$ mean of modeled wet day precipitation for  time period to bias correct
* $\overline{wetM_{bc}}$ bias corrected mean of modeled wet day precipitation for  time period to bias correct
* $\overline{O_{base}}$ mean of observations for historical base time period
* $\overline{M_{base}}$ mean of modeled values for base time period
* $\overline{M_{fut}}$ mean of modeled values for  time period to bias correct
* $\overline{M_{bc}}$ bias corrected mean of modeled values  for  time period to bias correct
* $\delta O_{base}$ observed anomalies for historical base time period
* $\delta M_{base}$ modeled anomalies for base time period
* $\delta M_{fut}$ modeled anomalies for  time period to bias correct
* $\delta M_{bc}$ bias corrected anomalies for time period to bias correct
* $\Delta $ predicted change in value at a given quantile 

## General Steps

1. Set training base period (~30 year period) for which there are both observed and modeled values. Example: 1976-2005.
2. Set time periods to bias correct individually (~30 year periods). Example: 1976-2005, 2006-2039, 2040-2069, 2070-2099
3. Loop through all time periods to bias correct
    1. Apply bias correction to a single time period
        1. Apply bias correction on a moving window. Loop through all days-of-year (366 total)
            1. Apply bias correction to a single day-of-year
                1. Get all observations and modeled values from base period that are within 15 days of the day-of-year (31-day moving window)
                2. Get all modeled values from time period to bias correct that are within 15 days of the day-of-year (31-day moving window) 
                3. Run **specified bias correction method** for day-of-year using *1.* and *2.* as input
        2. Run **specified bias correction method** over entire time period to bias correct without a moving window
        3. Bias correct *a.* with *b.* using specified bias correction method. This is the final bias corrected output for the time period.

## Precipitation Bias Correction Method
1. Bias correct number of wet days
    1. Find wet day threshold, $W_t$, such that applying $W_t$ makes the model’s number of wet days match the number of observed wet days over the training base period if all observations at or below $W_t$ are set to 0-mm. This fixes wet biases (too many wet days), but cannot adjust for dry biases (not enough wet days). If a model is dry biased, $W_t$ is set to 0-mm. 
    2. Set all model precipitation values $\leq W_t$ to 0-mm in both $M_{base}$ and $M_{fut}$ time periods.
2. Bias correct daily wet day precipitation totals
    1. Subset all time series to only wet day observations to create: $wetO_{base}, wetM_{base}, wetM_{fut}$
    2. Calculate mean values of $wetO_{base}, wetM_{base}, wetM_{fut}$ : $\overline{wetO_{base}}, \overline{wetM_{base}}, \overline{wetM_{fut}}$
    3. Calculate ratio anomalies, $\delta wetO_{base}, \delta wetM_{base}, \delta wetM_{fut}$ for each wet day time series using their respective mean values, $\overline{wetO_{base}}, \overline{wetM_{base}}, \overline{wetM_{fut}}$ , as the base values.
    4. Apply **equidistant quantile matching** to $\delta wetM_{fut}$ using $\delta wetO_{base}$ and $\delta wetM_{base}$ to create bias corrected anomaly time series, $\delta wetM_{bc}$ 
    5. Ensure that all $\delta wetM_{bc}$ values are positive. Set any $\delta wetM_{bc}$ values that are $\leq 0$ to the minimum positive $\delta wetM_{bc}$ value  
    6. Multiply $\delta wetM_{bc}$ by $K$ correction factor to ensure that the mean of $\delta wetM_{bc} = 1$ and the model’s predicted change in mean wet day precipitation is preserved. If the additive deltas applied by equidistant quantile matching are significantly skewed, the mean of $\delta wetM_{bc}$ might be slightly different from 1. This $K$ correction factor is analogous to the $K$ correction factor in ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)) section 3b.
        1. $K = \frac{1}{\delta wetM_{bc}}$
        2. $\delta wetM_{bc} = \delta wetM_{bc}\times K$
    7. Create bias corrected version of $\overline{wetM_{fut}}$ by multiplying by the ratio of $\overline{wetO_{base}}$ and $\overline{wetM_{base}}$
        1. $\overline{wetM_{bc}} = \delta wetM_{fut}\times (\frac{\overline{wetO_{base}}}{\overline{wetM_{base}}})$
    8. Multiply $\delta wetM_{bc}$ by $\overline{wetM_{bc}}$ to create bias corrected time series of wet day daily precipitation values, $wetM_{bc}$
3. Create final complete bias corrected daily precipitation time series, $M_{bc}$
    1. Combine $wetM_{bc}$ with zero precipitation days in $M_{fut}$ to create $M_{bc}$
    2. Calculate mean values of complete $O_{base}$ , $M_{base}$ , $M_{fut}$ , time series: $\overline{O_{base}}, \overline{M_{base}}, \overline{M_{fut}}$
    3. Create bias corrected version of $\overline{M_{fut}}$ by multiplying by the ratio of $\overline{O_{base}}$ and $\overline{M_{base}}$
        1. $\overline{M_{bc}} = M_{fut}\times \frac{\overline{O_{base}}}{\overline{M_{base}}}$
        2. Multiply ${M_{bc}}$ by $K$ correction factor to ensure that mean of ${M_{bc}}$, $\sim \overline{M_{bc}}$ , = $\overline{M_{bc}}$ and the model’s predicted change in mean precipitation is preserved. $\sim \overline{M_{bc}}$ should already be identical to $\overline{M_{bc}}$ if the wet day bias correction resulted in the number of wet days in $O_{base}$ equaling the number of wet days in $M_{base}$. If not (e.g.--model was dry biased with not enough wet days), the mean of $\sim \overline{M_{bc}}$ will not = $\overline{M_{bc}}$ and ${M_{bc}}$ needs to be adjusted. Note: in these cases, this correction results in the overall precipitation mean being correctly bias corrected at the expense of the wet day mean being incorrect. In other words, models with too many dry days will have unbiased total precipitation, but a biased combination of intensity and frequency.
            1. $K = \frac{\overline{M_{bc}}}{\sim \overline{M_{bc}}}$
            2. ${M_{bc}} = M_{bc}\times K$

## Temperature Bias Correction Method

1. Calculate mean values of  $O_{base}$ , $M_{base}$ , $M_{fut}$ : $\overline{O_{base}}$, $\overline{M_{base}}$, $\overline{M_{fut}}$
2. Calculate additive anomalies, $\delta O_{base}$ , $\delta M_{base}$ , $\delta M_{fut}$, for each wet time series using their respective mean values, $\overline{O_{base}}$, $\overline{M_{base}}$, $\overline{M_{fut}}$, as the base values. 
3. Apply **equidistant quantile matching** to $\delta M_{fut}$ using $\delta O_{base}$ and $\delta M_{base}$ to create bias corrected anomaly time series, $\delta M_{bc}$
4. Add $K$ correction factor to  $\delta M_{bc}$  to ensure that the mean of $\delta M_{bc}$  = 0 and the model’s predicted change in mean temperature is preserved. If the additive deltas applied by equidistant quantile matching are significantly skewed, the mean of $\delta M_{bc}$ might be slightly different from 0. This $K$ correction factor is analogous to the $K$ correction factor in ([Pierce et al. 2015](http://dx.doi.org/10.1175/JHM-D-14-0236.1)) section 3b.
    1. $K =  0 - \overline{\delta M_{bc}}$ 
    2. $\delta M_{bc} = \delta M_{bc} + K$
5. Create bias corrected version of $\overline{M_{fut}}$ by adding the difference of $\overline{O_{base}}$ and $\overline{M_{base}}$
    1. $\overline{M_{bc}} = \overline{M_{fut}} + ( \overline{O_{base}} - \overline{M_{base}})$
    2. Add $\overline{M_{bc}}$ to $\delta M_{bc}$ to get final bias corrected time series of daily temperature values, $M_{bc}$


## Equidistant Quantile Matching

1. Calculate empirical cumulative distribution function for training base period observation anomalies, $\delta O_{base}$, training base period modeled anomalies, $\delta M_{base}$, and modeled anomalies from time period to bias correct, $\delta M_{fut}$.
2. Map each value in $\delta M_{fut}$ to corresponding value in $\delta M_{base}$ based on percentile. For instance, the 45th percentile value in $\delta M_{fut}$ is mapped to the 45th percentile value in $\delta M_{base}$
3. Get difference, $\Delta$, for each value mapping. For both temperature and precipitation anomalies, $\Delta$ is an additive difference, $\delta M_{fut} - \delta M_{base}$.
4. Map each value in $\delta M_{fut}$ to corresponding value in $\delta O_{base}$ based on percentile. 
5. Apply corresponding $\Delta$ for each $\delta M_{fut}$ to corresponding mapped $\delta O_{base}$ value to get biased corrected anomalies, $\delta M_{bc}$. For both temperature and precipitation anomalies, $\delta M_{bc} = \delta O_{base} + \Delta$. 